# Тема “Создание признакового пространства”

## Продолжим обработку данных с Твиттера. 

Задания:
<ol>
<li><a href="#task_1">Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.<ul>
<li>Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
<li>Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
<li>Исключим стоп-слова с помощью stop_words='english'. 
<li>Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().</ul></a>  
<li><a href = "#task_2">Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.<ul>
<li>Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
<li>Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
<li>Исключим стоп-слова с помощью stop_words='english'.
<li>Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().</ul></a>
<li><a href = "#task_3">Натренируем gensim.models.Word2Vec модель на наших данных.<ul>
<li>Тренировать будем на токенизированных твитах combine_df['tweet_token']
<li>Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
<li>Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.</ul></a>
<li><a href = "#task_4">Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".</a>
<li><a href = "#task_5">Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
Давайте проверим векторное представление любого слова из нашего корпуса, например "food".</a>
<li><a href = "#task_6">Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.
Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.
На выходе должен получиться wordvec_df.shape = (49159, 200).</a>
</ol>

### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import pickle

from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


/home/oleg_rev/anaconda3/envs/base3_8/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


### Загрузка данных

In [2]:
with open('../Lesson1.Text_preprocessing/preprocessing_tweets_hw1.pkl', 'rb') as f:
    data = pickle.load(f)

data.head()

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


### Выполнение заданий

<p><a name="task_1"></a></p>

#### 1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
<ul>
<li>Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
<li>Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
<li>Исключим стоп-слова с помощью stop_words='english'. 
<li>Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().
</ul>


In [3]:
CountVectorizer?

Init signature:
CountVectorizer(
    *,
    input='content',
    encoding='utf-8',
    decode_error='strict',
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    analyzer='word',
    max_df=1.0,
    min_df=1,
    max_features=None,
    vocabulary=None,
    binary=False,
    dtype=<class 'numpy.int64'>,
)
Docstring:     
Convert a collection of text documents to a matrix of token counts

This implementation produces a sparse representation of the counts using
scipy.sparse.csr_matrix.

If you do not provide an a-priori dictionary and you do not use an analyzer
that does some kind of feature selection then the number of features will
be equal to the vocabulary size found by analyzing the data.

Read more in the :ref:`User Guide <text_feature_extraction>`.

Parameters
----------
input : {'filename', 'file', 'content'}, default='content'
    - If `'filename'`, the seque

In [4]:
# CountVectorizer(ngram_range=(1, 2), analyzer='word', binary=False,)
count_vectorizer = CountVectorizer(ngram_range=(1, 2),
                                   analyzer='word',
                                   preprocessor=' '.join,
                                   binary=False,
                                   max_df=0.9,
                                   max_features=1000,
                                   stop_words='english')

# Создаем the Bag-of-Words модель bag_of_words_tweet_stemmed
bag_of_words_tweet_stemmed = count_vectorizer.fit_transform(data['tweet_stemmed'])    # .apply(lambda x: ' '.join(x)))
# Создаем the Bag-of-Words модель bag_of_words_tweet_lemmatized
bag_of_words_tweet_lemmatized = count_vectorizer.fit_transform(data['tweet_lemmatized'])    # .apply(lambda x: ' '.join(x)))

# Отобразим Bag-of-Words модель как DataFrame
feature_names = count_vectorizer.get_feature_names()

In [5]:
pd.DataFrame(bag_of_words_tweet_stemmed.toarray(), columns = feature_names)


,able,absolutely,account,act,action,actor,actually,adapt,adapt environment,add,...,year,yes,yesterday,yo,yoga,york,young,youtube,yr,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
pd.DataFrame(bag_of_words_tweet_lemmatized.toarray(), columns = feature_names)


,able,absolutely,account,act,action,actor,actually,adapt,adapt environment,add,...,year,yes,yesterday,yo,yoga,york,young,youtube,yr,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<p><a name="task_2"></a></p>

#### 2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
<ul>
<li>Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
<li>Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
<li>Исключим стоп-слова с помощью stop_words='english'.
<li>Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().
</ul>

In [7]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.9,
                                   max_features=1000,
                                   preprocessor=' '.join,
                                   stop_words='english')

values_tweet_stemmed = tfidf_vectorizer.fit_transform(data['tweet_stemmed'])    # .apply(lambda x: ' '.join(x)))
values_tweet_lemmatized = tfidf_vectorizer.fit_transform(data['tweet_lemmatized'])    # .apply(lambda x: ' '.join(x)))


# Show the Model as a pandas DataFrame
feature_names = tfidf_vectorizer.get_feature_names()

In [8]:
pd.DataFrame(values_tweet_stemmed.toarray(), columns = feature_names)


,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,year,yes,yesterday,yo,yoga,york,young,youtube,yr,yummy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
pd.DataFrame(values_tweet_lemmatized.toarray(), columns = feature_names)


,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,year,yes,yesterday,yo,yoga,york,young,youtube,yr,yummy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<p><a name="task_3"></a></p>

#### 3. Натренируем gensim.models.Word2Vec модель на наших данных.
<ul><li>Тренировать будем на токенизированных твитах combine_df['tweet_token']
<li>Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
<li>Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.</ul>

In [10]:
Word2Vec?

Init signature:
Word2Vec(
    sentences=None,
    corpus_file=None,
    vector_size=100,
    alpha=0.025,
    window=5,
    min_count=5,
    max_vocab_size=None,
    sample=0.001,
    seed=1,
    workers=3,
    min_alpha=0.0001,
    sg=0,
    hs=0,
    negative=5,
    ns_exponent=0.75,
    cbow_mean=1,
    hashfxn=<built-in function hash>,
    epochs=5,
    null_word=0,
    trim_rule=None,
    sorted_vocab=1,
    batch_words=10000,
    compute_loss=False,
    callbacks=(),
    comment=None,
    max_final_vocab=None,
)
Docstring:     
Serialize/deserialize objects from disk, by equipping them with the `save()` / `load()` methods.

Warnings
--------
This uses pickle internally (among other techniques), so objects must not contain unpicklable attributes
such as lambda functions etc.
Init docstring:
Train, use and evaluate neural networks described in https://code.google.com/p/word2vec/.

Once you're finished training a model (=no more updates, only querying)
store and use only the :class:

In [11]:
modelW2V_tweet_token = Word2Vec(data['tweet_token'].tolist(),
                                vector_size=200,
                                window=5,
                                min_count=2,
                                sg=1,
                                negative=10,
                                workers=21,
                                seed=34)


In [12]:
modelW2V_tweet_token.build_vocab(data['tweet_token'].tolist())
modelW2V_tweet_token.train(data['tweet_token'].tolist(),
                           total_examples=modelW2V_tweet_token.corpus_count,
                           epochs=20)

(9024922, 11660880)

<p><a name="task_4"></a></p>

#### 4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. 
>Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".

In [13]:
modelW2V_tweet_token.wv.most_similar(positive='dinner')

[('stua', 0.5815574526786804),
 ('bihdaydinner', 0.5722419619560242),
 ('burritos', 0.5611501932144165),
 ('bolognese', 0.554709792137146),
 ('cookout', 0.5529578924179077),
 ('moes', 0.5519721508026123),
 ('hamburger', 0.5434655547142029),
 ('spaghetti', 0.5354280471801758),
 ('sissy', 0.5351146459579468),
 ('enroute', 0.5344194769859314)]

In [14]:
modelW2V_tweet_token.wv.most_similar(positive='trump')

[('blooms', 0.5866230130195618),
 ('donald', 0.5770735144615173),
 ('suppoer', 0.5731576085090637),
 ('trumpsamerica', 0.5608921051025391),
 ('unfit', 0.5584656596183777),
 ('carl', 0.5565658807754517),
 ('paladino', 0.5502210855484009),
 ('ally', 0.5441152453422546),
 ('form', 0.5424993634223938),
 ('unstable', 0.5389218330383301)]

<p><a name="task_5"></a></p>

#### 5. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
>Давайте проверим векторное представление любого слова из нашего корпуса, например "food".

In [15]:
vector = modelW2V_tweet_token.wv['food']  # get numpy vector of a word
print(f'vector shape {vector.shape}')
vector

vector shape (200,)


array([ 0.9950138 , -0.29494715,  0.311033  , -0.3220767 , -0.7961221 ,
        0.34122604, -0.594303  , -0.1964457 ,  0.5461771 ,  0.4517845 ,
        0.10287362,  0.11146805, -0.5638488 ,  0.25117877, -0.4548417 ,
       -0.08224328, -0.18303823, -0.22634684, -0.690878  , -0.12400456,
       -0.0020193 , -0.38965377, -0.92751294, -0.03035342, -0.42878255,
        0.5152175 , -0.33610684, -0.47148   ,  0.57660073, -0.15632625,
        0.54236144, -0.5051562 ,  0.20274453,  0.42110786, -0.6416487 ,
       -0.22645254, -0.34140897,  0.10189278,  0.10101406,  0.1480877 ,
       -0.20151944, -0.07603071,  0.6020438 ,  0.20290048,  0.06693158,
        0.23738024,  0.8132429 , -0.37625316, -0.15295443,  0.07205509,
       -0.42871577,  0.7332445 ,  0.24712546, -0.04229152,  0.958786  ,
        0.05209355, -0.12817399, -0.18582192,  0.25172967,  0.0877766 ,
       -0.3497725 ,  0.07612623, -0.21625711, -0.24771002, -0.17613265,
       -0.02344431,  0.6339707 ,  0.5203778 , -0.34171122, -0.36

<p><a name="task_6"></a></p>

#### 6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.
> Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.
На выходе должен получиться wordvec_df.shape = (49159, 200). 

In [16]:
@np.vectorize
def get_vec_tweet(tweet: list, model:Word2Vec, size: int):
    tweet_size = len(tweet)
    #vec=np.zeros((1, size))
    
    vec = model.wv[tweet]
    #print(vec/tweet_size)
    return vec/tweet_size


def _get_vec_tweet(tweet: list, model:Word2Vec, size: int, vec=np.zeros((1, 200))):
    tweet_size = len(tweet)
    #vec=np.zeros((1, size))
    print(tweet)
    for word in tweet:
        #print(tweet)
        print(word)
        vec += model.wv[word].reshape((1, size)) 
    #print(vec/tweet_size)
    return vec/tweet_size

In [17]:
for i in range(data['tweet_token'].shape[0]):
    _get_vec_tweet(tweet=data['tweet_token'][i],
              model=modelW2V_tweet_token,
              size=modelW2V_tweet_token.wv['food'].shape[0])

['when', 'father', 'is', 'dysfunctional', 'and', 'is', 'so', 'selfish', 'he', 'drags', 'his', 'kids', 'into', 'his', 'dysfunction', 'run']
when
father
is
dysfunctional
and
is
so
selfish
he
drags
his
kids
into
his
dysfunction
run
['thanks', 'for', 'lyft', 'credit', 'can', 'not', 'use', 'cause', 'they', 'do', 'not', 'offer', 'wheelchair', 'vans', 'in', 'pdx', 'disapointed', 'getthanked']
thanks
for
lyft
credit
can
not
use
cause
they
do
not
offer
wheelchair
vans
in
pdx
disapointed


KeyError: "Key 'disapointed' not present"

In [18]:
get_vec_tweet(tweet=data['tweet_token'],
              model=modelW2V_tweet_token,
              size=modelW2V_tweet_token.wv['food'].shape[0])

KeyError: "Key 'disapointed' not present"